In [1]:
import warnings
from datetime import datetime, timedelta
import numpy as np
import pandas as pd

import gc
import os
from tqdm import tqdm

from sklearn.model_selection import StratifiedKFold

warnings.simplefilter('ignore')
tqdm.pandas()
%matplotlib inline

pd.set_option('max_columns', None)
pd.set_option('max_rows', None)
pd.set_option('max_colwidth', 200)

In [2]:
seed = 2020

In [3]:
# !pip install lightgbm

In [4]:
df_train = pd.read_csv('raw_data/train.csv')
df_test = pd.read_csv('raw_data/testA.csv')

df_feature = df_train.append(df_test)
df_feature = df_feature.reset_index(drop=True)

In [5]:
df_feature.head()

,annualIncome,applicationType,delinquency_2years,dti,earliesCreditLine,employmentLength,employmentTitle,ficoRangeHigh,ficoRangeLow,grade,homeOwnership,id,initialListStatus,installment,interestRate,isDefault,issueDate,loanAmnt,n0,n1,n10,n11,n12,n13,n14,n2,n3,n4,n5,n6,n7,n8,n9,openAcc,policyCode,postCode,pubRec,pubRecBankruptcies,purpose,regionCode,revolBal,revolUtil,subGrade,term,title,totalAcc,verificationStatus
0,110000.0,0,0.0,17.05,Aug-2001,2 years,320.0,734.0,730.0,E,2,0,0,917.97,19.52,1.0,2014-07-01,35000.0,0.0,2.0,7.0,0.0,0.0,0.0,2.0,2.0,2.0,4.0,9.0,8.0,4.0,12.0,2.0,7.0,1.0,137.0,0.0,0.0,1,32,24178.0,48.9,E2,5,1.0,27.0,2
1,46000.0,0,0.0,27.83,May-2002,5 years,219843.0,704.0,700.0,D,0,1,1,461.90,18.49,0.0,2012-08-01,18000.0,NaN,NaN,13.0,NaN,NaN,NaN,NaN,NaN,NaN,10.0,NaN,NaN,NaN,NaN,NaN,13.0,1.0,156.0,0.0,0.0,0,18,15096.0,38.9,D2,5,1723.0,18.0,2
2,74000.0,0,0.0,22.77,May-2006,8 years,31698.0,679.0,675.0,D,0,2,0,298.17,16.99,0.0,2015-10-01,12000.0,0.0,0.0,11.0,0.0,0.0,0.0,4.0,3.0,3.0,0.0,0.0,21.0,4.0,5.0,3.0,11.0,1.0,337.0,0.0,0.0,0,14,4606.0,51.8,D3,5,0.0,27.0,2
3,118000.0,0,0.0,17.21,May-1999,10+ years,46854.0,689.0,685.0,A,1,3,1,340.96,7.26,0.0,2015-08-01,11000.0,6.0,4.0,9.0,0.0,0.0,0.0,1.0,6.0,6.0,4.0,16.0,4.0,7.0,21.0,6.0,9.0,1.0,148.0,0.0,0.0,4,11,9948.0,52.6,A4,3,4.0,28.0,1
4,29000.0,0,0.0,32.16,Aug-1977,NaN,54.0,694.0,690.0,C,1,4,0,101.07,12.99,0.0,2016-03-01,3000.0,1.0,2.0,12.0,0.0,0.0,0.0,4.0,7.0,7.0,2.0,4.0,9.0,10.0,15.0,7.0,12.0,1.0,301.0,0.0,0.0,10,21,2942.0,32.0,C2,3,11.0,27.0,2


In [6]:
df_feature.isDefault.value_counts()

0.0    640390
1.0    159610
Name: isDefault, dtype: int64

# 特征工程

In [7]:
del df_feature['policyCode']

In [8]:
grade_map = {'A': 1, 'B': 2, 'C': 3, 'D': 4, 'E': 5, 'F': 6, 'G': 7}
df_feature['grade'] = df_feature['grade'].map(grade_map)


def subGrade_map(x):
    grade, num = list(x)
    ans = grade_map[grade]
    ans = ans * 5 + int(num) - 1
    return ans


df_feature['subGrade'] = df_feature['subGrade'].map(subGrade_map)

In [9]:
def employmentLength_to_int(s):
    if pd.isnull(s):
        return s
    else:
        return np.int8(s.split()[0])


df_feature['employmentLength'].replace(
    to_replace='10+ years', value='10 years', inplace=True)
df_feature['employmentLength'].replace('< 1 year', '0 years', inplace=True)
df_feature['employmentLength'] = df_feature['employmentLength'].apply(
    employmentLength_to_int)

In [10]:
df_feature['issueDate_dt'] = pd.to_datetime(
    df_feature['issueDate'], format='%Y-%m-%d')
df_feature['issueDate_year'] = df_feature['issueDate_dt'].dt.year

df_feature['earliesCreditLine_year'] = df_feature['earliesCreditLine'].str.split(
    '-', expand=True)[1]
df_feature['earliesCreditLine_year'] = df_feature['earliesCreditLine_year'].astype(
    'int')

df_feature['issueDate_year_earliesCreditLine_year_minus'] = df_feature['issueDate_year'] - \
    df_feature['earliesCreditLine_year']

del df_feature['issueDate_dt']

In [11]:
df_feature['debt_ratio_year'] = df_feature['loanAmnt'] / df_feature['term'] / df_feature['annualIncome']

In [12]:
cate_features = ['applicationType', 'employmentLength', 'employmentTitle', 'grade', 'homeOwnership', 'initialListStatus',
                 'postCode', 'purpose', 'regionCode', 'subGrade', 'title', 'verificationStatus']
dense_features = ['annualIncome', 'delinquency_2years', 'dti', 'employmentLength', 'ficoRangeHigh',
                  'ficoRangeLow', 'installment', 'interestRate', 'loanAmnt', 'openAcc', 'pubRec', 'pubRecBankruptcies',
                  'revolBal', 'revolUtil', 'subGrade', 'term', 'totalAcc']

In [13]:
for f in tqdm(cate_features):
    df_feature['{}_cnt'.format(f)] = df_feature.groupby([f])[
        f].transform('count')

for f1 in tqdm(cate_features):
    for f2 in cate_features:
        if f1 != f2:
            df_feature['{}_{}_cnt'.format(f1, f2)] = df_feature.groupby([f1, f2])[
                f].transform('count')

100%|█████████████████████████████████████████████████████████████████████████████████████| 12/12 [00:41<00:00,  3.46s/it]


In [14]:
# 欺诈率
gps = []
for f in cate_features:
    gps.append([f])

for f1 in cate_features:
    for f2 in cate_features:
        if f1 != f2:
            gps.append([f1, f2])
                        
def statis_feat(df_know, df_unknow):
    for group_by in tqdm(gps):
        group = df_know.groupby(group_by).agg({'isDefault': ['mean']})
        columns = ['{}_default_ratio'.format('_'.join(group_by))]
        group.columns = columns
        group.reset_index(inplace=True)
        df_unknow = df_unknow.merge(group, on=group_by, how='left')

    return df_unknow


df_train = df_feature[~df_feature['isDefault'].isnull()]
df_train = df_train.reset_index(drop=True)
df_test = df_feature[df_feature['isDefault'].isnull()]

df_stas_feat = None
kf = StratifiedKFold(n_splits=5, random_state=seed, shuffle=True)
for train_index, val_index in kf.split(df_train, df_train['isDefault']):
    df_fold_train = df_train.iloc[train_index]
    df_fold_val = df_train.iloc[val_index]

    df_fold_val = statis_feat(df_fold_train, df_fold_val)
    df_stas_feat = pd.concat([df_stas_feat, df_fold_val], axis=0)

    del (df_fold_train)
    del (df_fold_val)
    gc.collect()

df_test = statis_feat(df_train, df_test)
df_feature = pd.concat([df_stas_feat, df_test], axis=0)
df_feature = df_feature.reset_index(drop=True)

del (df_stas_feat)
del (df_train)
del (df_test)
gc.collect()

100%|███████████████████████████████████████████████████████████████████████████████████| 144/144 [00:35<00:00,  4.04it/s]


23

In [15]:
df_feature.head()

,annualIncome,applicationType,delinquency_2years,dti,earliesCreditLine,employmentLength,employmentTitle,ficoRangeHigh,ficoRangeLow,grade,homeOwnership,id,initialListStatus,installment,interestRate,isDefault,issueDate,loanAmnt,n0,n1,n10,n11,n12,n13,n14,n2,n3,n4,n5,n6,n7,n8,n9,openAcc,postCode,pubRec,pubRecBankruptcies,purpose,regionCode,revolBal,revolUtil,subGrade,term,title,totalAcc,verificationStatus,issueDate_year,earliesCreditLine_year,issueDate_year_earliesCreditLine_year_minus,debt_ratio_year,applicationType_cnt,employmentLength_cnt,employmentTitle_cnt,grade_cnt,homeOwnership_cnt,initialListStatus_cnt,postCode_cnt,purpose_cnt,regionCode_cnt,subGrade_cnt,title_cnt,verificationStatus_cnt,applicationType_employmentLength_cnt,applicationType_employmentTitle_cnt,applicationType_grade_cnt,applicationType_homeOwnership_cnt,applicationType_initialListStatus_cnt,applicationType_postCode_cnt,applicationType_purpose_cnt,applicationType_regionCode_cnt,applicationType_subGrade_cnt,applicationType_title_cnt,applicationType_verificationStatus_cnt,employmentLength_applicationType_cnt,employmentLength_employmentTitle_cnt,employmentLength_grade_cnt,employmentLength_homeOwnership_cnt,employmentLength_initialListStatus_cnt,employmentLength_postCode_cnt,employmentLength_purpose_cnt,employmentLength_regionCode_cnt,employmentLength_subGrade_cnt,employmentLength_title_cnt,employmentLength_verificationStatus_cnt,employmentTitle_applicationType_cnt,employmentTitle_employmentLength_cnt,employmentTitle_grade_cnt,employmentTitle_homeOwnership_cnt,employmentTitle_initialListStatus_cnt,employmentTitle_postCode_cnt,employmentTitle_purpose_cnt,employmentTitle_regionCode_cnt,employmentTitle_subGrade_cnt,employmentTitle_title_cnt,employmentTitle_verificationStatus_cnt,grade_applicationType_cnt,grade_employmentLength_cnt,grade_employmentTitle_cnt,grade_homeOwnership_cnt,grade_initialListStatus_cnt,grade_postCode_cnt,grade_purpose_cnt,grade_regionCode_cnt,grade_subGrade_cnt,grade_title_cnt,grade_verificationStatus_cnt,homeOwnership_applicationType_cnt,homeOwnership_employmentLength_cnt,homeOwnership_employmentTitle_cnt,homeOwnership_grade_cnt,homeOwnership_initialListStatus_cnt,homeOwnership_postCode_cnt,homeOwnership_purpose_cnt,homeOwnership_regionCode_cnt,homeOwnership_subGrade_cnt,homeOwnership_title_cnt,homeOwnership_verificationStatus_cnt,initialListStatus_applicationType_cnt,initialListStatus_employmentLength_cnt,initialListStatus_employmentTitle_cnt,initialListStatus_grade_cnt,initialListStatus_homeOwnership_cnt,initialListStatus_postCode_cnt,initialListStatus_purpose_cnt,initialListStatus_regionCode_cnt,initialListStatus_subGrade_cnt,initialListStatus_title_cnt,initialListStatus_verificationStatus_cnt,postCode_applicationType_cnt,postCode_employmentLength_cnt,postCode_employmentTitle_cnt,postCode_grade_cnt,postCode_homeOwnership_cnt,postCode_initialListStatus_cnt,postCode_purpose_cnt,postCode_regionCode_cnt,postCode_subGrade_cnt,postCode_title_cnt,postCode_verificationStatus_cnt,purpose_applicationType_cnt,purpose_employmentLength_cnt,purpose_employmentTitle_cnt,purpose_grade_cnt,purpose_homeOwnership_cnt,purpose_initialListStatus_cnt,purpose_postCode_cnt,purpose_regionCode_cnt,purpose_subGrade_cnt,purpose_title_cnt,purpose_verificationStatus_cnt,regionCode_applicationType_cnt,regionCode_employmentLength_cnt,regionCode_employmentTitle_cnt,regionCode_grade_cnt,regionCode_homeOwnership_cnt,regionCode_initialListStatus_cnt,regionCode_postCode_cnt,regionCode_purpose_cnt,regionCode_subGrade_cnt,regionCode_title_cnt,regionCode_verificationStatus_cnt,subGrade_applicationType_cnt,subGrade_employmentLength_cnt,subGrade_employmentTitle_cnt,subGrade_grade_cnt,subGrade_homeOwnership_cnt,subGrade_initialListStatus_cnt,subGrade_postCode_cnt,subGrade_purpose_cnt,subGrade_regionCode_cnt,subGrade_title_cnt,subGrade_verificationStatus_cnt,title_applicationType_cnt,title_employmentLength_cnt,title_employmentTitle_cnt,title_grade_cnt,title_homeOwnership_cnt,title_initi

In [16]:
# # Function to reduce the memory usage
# def reduce_mem_usage(df, verbose=True):
#     numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
#     start_mem = df.memory_usage().sum() / 1024**2
#     for col in tqdm([f for f in df.columns if f not in ['query_time']]):
#         col_type = df[col].dtypes
#         if col_type in numerics:
#             c_min = df[col].min()
#             c_max = df[col].max()
#             if str(col_type)[:3] == 'int':
#                 if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(
#                         np.int8).max:
#                     df[col] = df[col].astype(np.int8)
#                 elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(
#                         np.int16).max:
#                     df[col] = df[col].astype(np.int16)
#                 elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(
#                         np.int32).max:
#                     df[col] = df[col].astype(np.int32)
#                 elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(
#                         np.int64).max:
#                     df[col] = df[col].astype(np.int64)
#             else:
#                 if c_min > np.finfo(np.float16).min and c_max < np.finfo(
#                         np.float16).max:
#                     df[col] = df[col].astype(np.float16)
#                 elif c_min > np.finfo(np.float32).min and c_max < np.finfo(
#                         np.float32).max:
#                     df[col] = df[col].astype(np.float32)
#                 else:
#                     df[col] = df[col].astype(np.float64)
#     end_mem = df.memory_usage().sum() / 1024**2
#     if verbose:
#         print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(
#             end_mem, 100 * (start_mem - end_mem) / start_mem))
#     return df

# df_feature = reduce_mem_usage(df_feature)

In [17]:
os.makedirs('data', exist_ok=True)
df_feature.to_pickle('data/feature.pkl')